# Estimación de dirección de arribo de fuentes sonoras 

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import gspread # Extension para leer el excel desde Drive
from oauth2client.service_account import ServiceAccountCredentials # Extensión para autentificación con Drive
from tdoa_py import estimation, simulation_room

### Nombre de la simulación

In [2]:
#Nombre de la simulación
simulation_name = "hoja_abel"

### Configuración de Gspreadsheet

En caso de no querer utilizar GSpreadsheet se puede armar un dataframe que contenga las siguientes columnas: 

sim_name	method	audio	room_x	room_y	room_z	rt60	snr_db	n_mics	mic_d	mic_z	mic_directivity	src_dist	src_z	src_az_deg	avg_angle_deg	theta_1	theta_2	theta_3	t_12	t_13	t_14	avg_tdoa	room_dim

In [3]:
# Configurar credenciales
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("simulationsdoa-credenciales.json", scope)
client = gspread.authorize(creds)

# Abrir la hoja de cálculo 
# Pegar link del GSpreadsheet
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/13XTDng98P99pfexK78Dd4Gud1CzZwO7PfVhpyIG1jCM/edit?gid=1166684204#gid=1166684204")
sheet = spreadsheet.worksheet(simulation_name)

# Leer datos
data = sheet.get_all_records()
df_simulations = pd.DataFrame(data)

df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1) # Para pasar el recinto en una tupla luego
df_simulations.head() # Verificar el resultado

,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,avg_angle_deg,angle_error,angle_error_pct,std_angle,angle_12,angle_13,angle_14,angle_22,angle_23,angle_32
0,sweep_roth_high_rev_0,roth,sine_sweep_24bit.wav,20.0,40.0,10.0,5,90,4,0.1,...,18.90,18.90,0.00,3.82,21.73,15.27,12.42,21.73,15.27,21.73
1,sweep_roth_high_rev_5,roth,sine_sweep_24bit.wav,20.0,40.0,10.0,5,90,4,0.1,...,18.90,13.90,277.99,3.82,21.73,15.27,12.42,21.73,15.27,21.73
2,sweep_roth_high_rev_10,roth,sine_sweep_24bit.wav,20.0,40.0,10.0,5,90,4,0.1,...,19.48,9.48,94.84,2.94,21.73,15.27,17.68,21.73,15.27,21.73
3,sweep_roth_high_rev_15,roth,sine_sweep_24bit.wav,20.0,40.0,10.0,5,90,4,0.1,...,21.28,6.28,41.85,1.51,21.73,21.73,17.68,21.73,21.73,21.73
4,sweep_roth_high_rev_20,roth,sine_sweep_24bit.wav,20.0,40.0,10.0,5,90,4,0.1,...,29.14,9.14,45.69,2.44,30.96,26.72,25.16,30.96,26.72,30.96


## Bloque de código para ejecutar todo directamente

Permite cargar los datos del Excel y procesa todo en tiempo real. Acá se puede elegir si guardar o no los audios con **save_audio = True**, en caso de ser falso se trabajará todo con numpy array con los datos de las señales procesadas.

In [ ]:
# Iterar sobre el DataFrame 
for idx, sim in df_simulations.iterrows():
    src_az_deg, mic_signals = simulation_room.sim_room_Nmics(
        wav_path=f'audios/anechoic/{sim["audio"]}',
        out_dir=f'audios/output/{simulation_name}/{sim["sim_name"]}',
        sim_name=sim["sim_name"],
        fs=48000,
        room_dim=sim["room_dim"],
        rt60=sim["rt60"],
        snr_db=sim["snr_db"],
        n_mics=sim["n_mics"],
        mic_d=sim["mic_d"],
        mic_z=sim["mic_z"],
        mic_directivity=sim["mic_directivity"],
        src_dist=sim["src_dist"],
        src_az_deg=sim["src_az_deg"],
        src_z=sim["src_z"],
        save_audio=True
    )
    
    avg_angle, avg_tdoa, angles_per_mic_ref, tdoas_per_mic_ref = estimation.estimate_doa(
        signals=mic_signals,
        fs=48000,
        d=sim["mic_d"],
        method=sim["method"]
    )
    
    # Guardar resultados en cada fila individual

    df_simulations.at[idx, 'avg_angle_deg'] = avg_angle
    df_simulations.at[idx, 'angle_error'] = np.abs(avg_angle - sim["src_az_deg"])
    df_simulations.at[idx, 'angle_error_pct'] = (df_simulations.at[idx, 'angle_error'] / sim["src_az_deg"]) * 100 if sim["src_az_deg"] != 0 else 0
    df_simulations.at[idx, 'std_angle'] = np.std([angle for angles in angles_per_mic_ref for angle in angles])

    for i in range(len(angles_per_mic_ref)):
        for j in range(len(angles_per_mic_ref[i])):
            df_simulations.at[idx, f'angle_{i+1}{j+2}'] = angles_per_mic_ref[i][j]
            
    
# # Convertir el DataFrame a una lista de listas para Google Sheets
df_simulations["room_dim"] = df_simulations["room_dim"].apply(lambda x: ",".join(map(str, x)))
data = [df_simulations.columns.values.tolist()] + df_simulations.values.tolist()



# Escribir en la hoja de cálculo
sheet.update(data)
print("DataFrame guardado correctamente en Google Sheets.")
df_simulations.head()



APIError: APIError: [400]: Unable to parse range: 'hoja_abel'